In [2]:
from src.datasets.nli import *
from src.model.nli_models import *
from src.utils.nli_utils import *

In [3]:
snli_conf = {"batch_size":128,"max_len":100,"device":'cuda',"tokenizer":'distil_bert',"use_char_emb":False,"max_word_len":10}
dataset = mnli_module(snli_conf)

In [4]:
dataset.prepare_data()


downloading multinli_1.0.zip
/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning:

Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning:

LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.

multinli_1.0.zip: 100%|██████████| 227M/227M [00:09<00:00, 24.4MB/s]
extracting
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:13: UserWarning:

Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning:

BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.



In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from transformers import DistilBertModel


class Bert_Encoder(nn.Module):
    def __init__(self):
        super(Bert_Encoder, self).__init__()
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')

    def forward(self, x0):
        enc, = self.bert.forward(x0)
        enc = torch.mean(enc,dim=1)
        # enc = torch.max(enc,dim=1).values
        return enc

class Bert_snli(nn.Module):
    def __init__(self,conf):
        super(Bert_snli,self).__init__()
        self.bert = Bert_Encoder()
        self.fc = nn.Linear(4*768,3)

    def forward(self,x0,x1):
        x0_enc = self.bert(x0)
        x1_enc = self.bert(x1)
        enc = torch.cat([x0_enc,x1_enc,torch.abs(x0_enc-x1_enc),x0_enc*x1_enc],dim=1)
        opt = self.fc(enc)
        return opt


In [14]:

model_conf = {}

hparams = {
    "optimizer_base": {
        "optim": "adamw",
        "lr": 0.0010039910781394373,
        "scheduler": "const",
    },
    "optimizer_tune": {
        "optim": "adam",
        "lr": 0.0010039910781394373,
        "weight_decay": 0.1,
        "scheduler": "lambda",
    },
    "switch_epoch": 5,
}

model = Bert_snli(model_conf)
# model = SNLI_model(attn_bilstm_snli,model_conf,hparams=hparams)

In [15]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
count_parameters(model)

66372099

In [16]:
train_dl,val_dl,test_dl = dataset.data.get_dataloaders()

In [17]:
for i in train_dl:
    print(i)
    break

[[tensor([[  101, 17158,  2135,  ...,     0,     0,     0],
        [  101,  2017,  2113,  ...,     0,     0,     0],
        [  101,  2028,  1997,  ...,     0,     0,     0],
        ...,
        [  101,  1996,  2773,  ...,     0,     0,     0],
        [  101,  1998,  1037,  ...,     0,     0,     0],
        [  101,  2426,  2122,  ...,     0,     0,     0]]), tensor([[ 101, 4031, 1998,  ...,    0,    0,    0],
        [ 101, 2017, 4558,  ...,    0,    0,    0],
        [ 101, 1037, 2266,  ...,    0,    0,    0],
        ...,
        [ 101, 1996, 2611,  ...,    0,    0,    0],
        [ 101, 2045, 3513,  ...,    0,    0,    0],
        [ 101, 1996, 6986,  ...,    0,    0,    0]])], tensor([1., 2., 2., 2., 1., 2., 1., 2., 0., 0., 2., 0., 1., 1., 0., 2., 2., 0.,
        0., 1., 2., 2., 0., 2., 2., 1., 0., 1., 0., 2., 2., 2., 0., 1., 0., 2.,
        2., 0., 0., 0., 2., 1., 2., 1., 2., 2., 0., 0., 1., 2., 2., 2., 0., 1.,
        1., 1., 0., 1., 0., 1., 1., 2., 0., 2., 1., 0., 0., 0., 0.,

In [21]:
model = model.cuda()

In [23]:
from torch_lr_finder import LRFinder

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
lr_finder = LRFinder(model, optimizer, criterion, device="cuda")
lr_finder.range_test(train_dl, val_loader=val_dl, end_lr=0.01, num_iter=100, step_mode="exp")
lr_finder.plot(log_lr=False)
lr_finder.reset()

TypeError: forward() missing 1 required positional argument: 'x1'